In [1]:
pip install transformers sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://repo.ai.gato/registry/repository/pypi-proxy/simple
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
# 모델 및 토크나이저 불러오기
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [21]:
# 텍스트=>정수 인코딩(1개의 문장)

input = tokenizer.encode("I evaluated the performance of GPT-Neo developed by OpenAI.", return_tensors="pt")
print(input[0])
print(tokenizer.decode(input[0]))

tensor([   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
         4166,   416,  4946, 20185,    13])
I evaluated the performance of GPT-Neo developed by OpenAI.


In [22]:
# 정수 인코딩(복수 문장)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
input = tokenizer.batch_encode_plus(
    ["I evaluated the performance of GPT-Neo developed by OpenAI.",
     "I evaluated the performance of GPT developed by OpenAI."]
     , padding=True, truncation=True, return_tensors="pt")

print(input['input_ids'])
print([tokenizer.decode(input['input_ids'][i]) for i in range(len(input['input_ids']))])

tensor([[   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
          4166,   416,  4946, 20185,    13],
        [   40, 16726,   262,  2854,   286,   402, 11571,  4166,   416,  4946,
         20185,    13, 50257, 50257, 50257]])
['I evaluated the performance of GPT-Neo developed by OpenAI.', 'I evaluated the performance of GPT developed by OpenAI. [PAD] [PAD] [PAD]']


In [24]:
# 복수의 문장 인코딩
input = tokenizer.batch_encode_plus(
    ["I evaluated the performance of GPT2 developed by OpenAI.",
     "Vaccine for new coronavirus in the UK",
    "3.1415926535"]
    , max_length=5, truncation=True, padding=True, return_tensors="pt")

In [25]:
# 인코딩 결과 확인
input['input_ids']

tensor([[   40, 16726,   262,  2854,   286],
        [   53,  4134,   500,   329,   649],
        [   18,    13,  1415, 19707, 22980]])

In [27]:
# 문장 만들기
generated = model.generate(input['input_ids'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [28]:
# 생성된 문장 디코딩
generated_text = tokenizer.batch_decode(generated)
for i, sentence in enumerate(generated_text):
  print(f'No.{i+1}')
  print(f"{sentence}\n")

No.1
I evaluated the performance of the proposed method on the real-world dataset. The results are shown in

No.2
Vaccine for new-borns

The vaccine for new-borns is a vaccine

No.3
3.1415926535897932384626433832795028841971693



In [29]:
# DistilGPT2 모형 활용
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [32]:
# DistilGPT2 모형으로 문장 만들기
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=12)
greedy_output = model.generate(input_ids, max_length=20)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good thing to have a good friend.






### DialoGPT 모형 활용

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [39]:
#문장 만들기
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=30)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

#좀더 자연스러운 문장이 만들어짐

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good way to get a feel for the game.


In [44]:
input_ids = tokenizer.encode("Covid19 delta is spreading", return_tensors='pt')
# greedy_output = model.generate(input_ids, max_length=10)
greedy_output = model.generate(input_ids, max_length=20)

print("Outputs:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Outputs:
----------------------------------------------------------------------------------------------------
Covid19 delta is spreading the word


In [45]:
# MLM(Masked Language Model): 문서의 일부를 가리고(mask) 원래 단어를 추측하는 빈칸 채우기

from transformers import pipeline

unmasker = pipeline('fill-mask', model='bert-base-uncased')

# [Mask]가 포함된 문장 입력
unmasker("I [MASK] apple.")
# [Mask] 위치에 입력될 수 있는 단어들을 추천하여 문장 완성

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.1829015463590622,
  'token': 2293,
  'token_str': 'love',
  'sequence': 'i love apple.'},
 {'score': 0.12624028325080872,
  'token': 2066,
  'token_str': 'like',
  'sequence': 'i like apple.'},
 {'score': 0.11780295521020889,
  'token': 2359,
  'token_str': 'wanted',
  'sequence': 'i wanted apple.'},
 {'score': 0.06842318177223206,
  'token': 2215,
  'token_str': 'want',
  'sequence': 'i want apple.'},
 {'score': 0.055607497692108154,
  'token': 3866,
  'token_str': 'loved',
  'sequence': 'i loved apple.'}]

In [46]:
# 제일 높게 출력된 단어 와 완성된 문장
# 'token_str': 'loved',
# 'sequence': 'i loved apple.'}]

In [49]:
from transformers import pipeline

# distilbert 모델
unmasker = pipeline('fill-mask', model = 'distilbert-base-uncased')
unmasker('I [MASK] apple.')

[{'score': 0.06269729137420654,
  'token': 8823,
  'token_str': 'ate',
  'sequence': 'i ate apple.'},
 {'score': 0.05864424630999565,
  'token': 2293,
  'token_str': 'love',
  'sequence': 'i love apple.'},
 {'score': 0.05670216679573059,
  'token': 3866,
  'token_str': 'loved',
  'sequence': 'i loved apple.'},
 {'score': 0.05136874318122864,
  'token': 6283,
  'token_str': 'hated',
  'sequence': 'i hated apple.'},
 {'score': 0.0491315983235836,
  'token': 4521,
  'token_str': 'eat',
  'sequence': 'i eat apple.'}]

In [50]:
# 모형에 따라 추천되는 단어가 달라짐
# 'token_str': 'eat',
# 'sequence': 'i eat apple.'}]

In [52]:
from transformers import pipeline

# albert 모델: bert 모델의 정확도를 높이고 경량화한 모형
unmasker = pipeline('fill-mask', model='albert-base-v2')
unmasker('I [MASK] apple.')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.12620007991790771,
  'token': 339,
  'token_str': 'love',
  'sequence': 'i love apple.'},
 {'score': 0.0922037661075592,
  'token': 3345,
  'token_str': 'liked',
  'sequence': 'i liked apple.'},
 {'score': 0.05626443028450012,
  'token': 2199,
  'token_str': 'loved',
  'sequence': 'i loved apple.'},
 {'score': 0.04441851004958153,
  'token': 5285,
  'token_str': 'hated',
  'sequence': 'i hated apple.'},
 {'score': 0.0399409718811512,
  'token': 3223,
  'token_str': 'hate',
  'sequence': 'i hate apple.'}]

In [ ]:
# 모델마다 다른 결과
# 'token_str': 'hate',
# 'sequence': 'i hate apple.'}]